# Tarea 2 - Mineria de Datos
## Árboles de Regresión
Leonardo Olivares

In [13]:
import pandas as pd
import numpy as np
from queue import Queue
from graphviz import Digraph

### Pre-Procesamiento de Datos

In [14]:
df = pd.read_csv('FATS_GAIA.dat')

In [15]:
msk = np.random.rand(len(df)) < 0.8
train_df = df[msk]
test_df = df[~msk]

In [16]:
target = "PeriodLS"

### Implementación del Algoritmo

In [17]:
class Node:
    
    id = 1
    
    def __init__(self, label=None, value=None, n_type=None):
        self.id = Node.id 
        self.left_child = None
        self.right_child = None
        self.label = label
        self.value = value
        self.type = n_type
        self.greater = {}
        self.leq = {}
        self.leafe = False
        self.parent_id = None
        self.height = 0
        
        Node.id += 1
        
    def set_node(self, train_df, target, max_depth):
        options = train_df.keys()
        
        variance_dict = dict()
        
        print(train_df[target].count(), "ANTES DE LOS CAMBIOS")
        total = train_df[target].count()
        for g in self.greater:
            print(g, self.greater[g], "GREATER")
            train_df = train_df[train_df[g] > self.greater[g]]
                
        for l in self.leq:
            print(l, self.leq[l], "LESS")
            train_df = train_df[train_df[l] <= self.leq[l]]
            
        
        print(train_df[target].count(), "Despues de LOS CAMBIOS")
        #target_variance = train_df[target].var()
        total_len = train_df[target].count()
        #print("TOTAL LEN {}".format(total_len))

        if max_depth and self.height >= max_depth:
            self.leafe = True
            self.value = np.mean(train_df[target])
            print("Este nodo es una hoja con valor {}".format(self.value))
        
        elif total_len >= 100:
            splits = dict()
            
            for option in options:              

                if option != target and option != "Class":

                    spl = self.best_split(train_df, option, target, total_len)
                    splits[option] = spl


                    df_leq = (train_df[train_df[option] <= spl])
                    df_greater = (train_df[train_df[option] > spl])
                    
                    count_leq = df_leq[target].count()
                    count_greater = df_greater[target].count()


                    option_variance = (np.var(df_leq[target]) * (count_leq/total_len)) + (np.var(df_greater[target]) * (count_greater/total_len))
                    variance_dict[option] = option_variance


            min_regressor = min(variance_dict, key=variance_dict.get)
            

            
            self.label = min_regressor
            self.value = splits[self.label]
            print("El nodo tiene label {} y valor de split {}".format(self.label, self.value))
        
        else:
            self.leafe = True
            self.value = np.mean(train_df[target])
            print("Este nodo es una hoja con valor {}".format(self.value))
            
            
    def best_split(self, train_df, option, target, total_len):
        perc = 0.1
        best = None
        last_try = None
        
        variance_options = dict()

        while perc < 0.9:
            
            split = train_df[option].quantile(perc)
            
            if last_try != split:

                df_leq = (train_df[train_df[option] <= split])
                df_greater = (train_df[train_df[option] > split])
                count_leq = df_leq[target].count()
                count_greater = df_greater[target].count()

                
                left_variance = np.var(df_leq[target]) * (count_leq/total_len)
                right_variance = np.var(df_greater[target]) * (count_greater/total_len)
                if left_variance and right_variance:
                    variance = left_variance + right_variance
                else:
                    variance = np.var(train_df[target])
                    
                variance_options[split] = variance

                if not best:
                    best = variance
                    last_try = split

                elif variance < best:
                    best = variance
                    last_try = split


            perc += 0.1
            perc = round(perc, 1)
            
        return min(variance_options, key=variance_options.get)

In [18]:
class RegressionTree:
    
    def __init__(self, target):
        self.root = Node()
        self.target = target
        
    def fit(self, train_df, target ,max_depth=None):
        
        nodes = Queue()
        nodes.put(self.root)
        
        while not nodes.empty():
            node = nodes.get()
            print("Analizando Nodo {} con padre {}".format(node.id, node.parent_id))
            print("Con altura {}".format(node.height))

            node.set_node(train_df, self.target, max_depth)
            
            if not node.leafe:
            
                node.left_child = Node()
                node.left_child.parent_id = node.id
                node.right_child = Node()
                node.right_child.parent_id = node.id
                
                node.left_child.height = node.height + 1
                node.right_child.height = node.height + 1

                node.left_child.leq = node.leq.copy()
                node.left_child.greater = node.greater.copy()
                
                
                node.right_child.greater = node.greater.copy()
                node.right_child.leq = node.leq.copy()
                
                
                node.left_child.leq[node.label] = node.value
                node.right_child.greater[node.label] = node.value

                nodes.put(node.left_child)
                nodes.put(node.right_child)
                
            
            print("  ")
            
            
    def predict(self, test_df, index, r_tree):
        node = r_tree.root
        row = test_df.iloc[index]
        while not node.leafe:

            if row[node.label] <= node.value:
                node = node.left_child
            else:
                node = node.right_child
        return node.value
    

        
        

### Aplicación del Algoritmo
#### Entrenamiento

In [19]:
r_tree = RegressionTree(target)
r_tree.fit(train_df, target, 10)

Analizando Nodo 1 con padre None
Con altura 0
160628 ANTES DE LOS CAMBIOS
160628 Despues de LOS CAMBIOS
El nodo tiene label Autocor_length y valor de split 2.0
  
Analizando Nodo 2 con padre 1
Con altura 1
160628 ANTES DE LOS CAMBIOS
Autocor_length 2.0 LESS
114099 Despues de LOS CAMBIOS
El nodo tiene label Autocor_length y valor de split 1.0
  
Analizando Nodo 3 con padre 1
Con altura 1
160628 ANTES DE LOS CAMBIOS
Autocor_length 2.0 GREATER
46529 Despues de LOS CAMBIOS
El nodo tiene label Psi_eta y valor de split 0.525012238
  
Analizando Nodo 4 con padre 2
Con altura 2
160628 ANTES DE LOS CAMBIOS
Autocor_length 1.0 LESS
87082 Despues de LOS CAMBIOS
El nodo tiene label MaxSlope y valor de split 2.236974007
  
Analizando Nodo 5 con padre 2
Con altura 2
160628 ANTES DE LOS CAMBIOS
Autocor_length 1.0 GREATER
Autocor_length 2.0 LESS
27017 Despues de LOS CAMBIOS
El nodo tiene label Freq3_harmonics_amplitude_0 y valor de split 0.12515999800000013
  
Analizando Nodo 6 con padre 3
Con altura 2

#### Predicciones y Accuracy

In [20]:
results_dif = list()
for row in range(len(test_df)):
    expected = test_df.iloc[row][target]
    prediction = r_tree.predict(test_df, row, r_tree)
    
    results_dif.append(expected-prediction)
    
var_y = np.var(test_df[target])
var_results = np.var(results_dif)
acc = 1 - (var_results/var_y)

print("Accuracy {}".format(acc))
    
    
    

Accuracy 0.6472437738915207


## Visualizacion del Árbol

In [26]:
dot = Digraph(comment='Tree', node_attr={'shape': 'box'})

nodes = [r_tree.root]
while nodes:
    node = nodes.pop(0)
    
    
    if not node.leafe:
        if not node.left_child.leafe:
            dot.edge(node.label+str(node.id), node.left_child.label+str(node.left_child.id), label="<=" + str(node.value))
        else:
            dot.edge(node.label+str(node.id), str(node.left_child.value), label="<=" + str(node.value))
        if not node.right_child.leafe:
            dot.edge(node.label+str(node.id), node.right_child.label+str(node.right_child.id), label=">" + str(node.value))
        else:
            dot.edge(node.label+str(node.id), str(node.right_child.value), label=">" + str(node.value))


    if node.left_child:
        nodes.append(node.left_child)
    if node.right_child:
        nodes.append(node.right_child)
    
    
dot
    

### Explicación y Conclusiones

El resultado obtenido mediante el árbol de regresión fue el de un accuracy de 0.6 aproximadamente, siendo 1 la representacion de accuracy perfecto. Este resultado puede que no sea el más ideal, debido a que el 40% de las veces se aleja del resultado esperado. Es por esto que, si se requiere de una regresion mas precisa, se puede aplicar el método de clasificación "Random Forest", a partir del cual mediante un mayor número de árboles, el valor se acerca más al real.

Para comprobar que el resultado obtenido era válido, se realizó una implementación por separado del árbol, mediante la librería sklearn, y los mejores accuracies fueron efectivamente de 0,6 aproximadamente.

Para conseguir el mejor accuracy posible, fue necesario realizar varias iteraciones alterando principalmente dos parámetros. El primero era la cantidad de registros requeridos para seguir agregando nodos, debido a que si puede afectar construir un nodo con pocos registros significativos, causando overfitting. Por otro lado, la altura del árbol se controló, ya que si se permite que el algoritmo continue expandiendo el árbol hacia abajo, se termina ajustando sobre lo deseado al set de entrenamiento, generando peores resultados.

Alterando estos valores, y comprobando con sklearn, los parametros ideales fueron:
* Cantidad mínima de registros por nodos: 100
* Altura máxima: 10

La mejor feature que permitía separar los datos en dos grupos, con la menor varianza posible, fue "Autocor_length", y por lo tanto, fue seleccionada por el algoritmo para ser la raíz del árbol.

Como se mencionó anteriormente, y se comprobó, aumentando la altura del árbol causa un sobre ajuste a los datos de entrenamiento, empeorando el accuracy al probar con datos distintos del set de testeo.

En cuanto a la implementación general del árbol, para encontrar el mejor split del nodo seleccionado, se registro las varianzas obtenidas al realizar la división en distintos percentiles, y seleccionando aquella que resultaba con una menor varianza del target.
